In [19]:
import numpy as np
import pandas as pd

import sklearn 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score

# Import plotting libraries
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt
import optuna
import sklearn.model_selection
import sklearn.preprocessing


# Set larger fontsize for all plots
matplotlib.rcParams.update({'font.size': 18})
from IPython.display import clear_output

### Load data

In [20]:
df = pd.read_csv('bg_struct.csv', index_col=0)

In [21]:
df.describe()

,bandgap,cell_length_a,cell_lentgh_b,cell_lentgh_c,cell_angle_alpha,cell_angle_beta,cell_angle_gamma
count,16335.000000,16335.000000,16335.000000,16335.000000,16335.000000,16335.000000,16335.000000
mean,2.413936,7.756010,8.213617,11.564255,84.577062,87.422030,87.855174
std,0.373129,3.181366,2.887233,9.437813,18.476919,16.642053,22.486535
min,1.800200,2.363000,2.363000,2.661370,5.200300,5.397478,5.397478
25%,2.088000,5.609011,6.203360,7.542867,74.139139,88.470574,86.084645
50%,2.389300,7.131163,7.760993,9.836234,90.000000,90.000000,90.000000
75%,2.728300,9.018695,9.606510,12.994967,90.000000,90.000000,91.056782
max,3.099700,38.477274,40.417878,211.425839,168.624134,168.624134,167.256937


#### Define x and y

In [24]:
X = df[['cell_length_a','cell_lentgh_b', 'cell_lentgh_c',
        'cell_angle_alpha', 'cell_angle_beta', 'cell_angle_gamma']]
y = df['bandgap']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True)

### Do hyperparamter tuning to optimize the training Decision Tree model

In [26]:
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 2, 80)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 40)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    splitter = trial.suggest_categorical('splitter',['best','random'])
    min_weight_fraction_leaf = trial.suggest_uniform(
        'min_weight_fraction_leaf', 0.0, 0.5)
    # get the score for the hyperparameters chosen
    regr = DecisionTreeRegressor(max_depth=max_depth, min_samples_split=min_samples_split,
                                 min_samples_leaf=min_samples_leaf,splitter=splitter,
                                 min_weight_fraction_leaf=min_weight_fraction_leaf)
    regr.fit(X_train, y_train)
    if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return regr.score(X_test, y_test)

study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize')
study.optimize(objective, n_trials=5000)
print('Best params: ', study.best_params)

[I 2022-03-13 23:20:34,537] A new study created in memory with name: no-name-9d26421c-1526-4f11-b045-ac37e6db2486
[I 2022-03-13 23:20:34,573] Trial 0 finished with value: 0.006301227937167475 and parameters: {'max_depth': 7, 'min_samples_split': 31, 'min_samples_leaf': 20, 'splitter': 'best', 'min_weight_fraction_leaf': 0.12232943497016546}. Best is trial 0 with value: 0.006301227937167475.
[I 2022-03-13 23:20:34,578] Trial 1 finished with value: -0.00043311595191020125 and parameters: {'max_depth': 21, 'min_samples_split': 36, 'min_samples_leaf': 14, 'splitter': 'random', 'min_weight_fraction_leaf': 0.4337802139969552}. Best is trial 0 with value: 0.006301227937167475.
[I 2022-03-13 23:20:34,584] Trial 2 finished with value: -0.00035520424272772466 and parameters: {'max_depth': 3, 'min_samples_split': 40, 'min_samples_leaf': 14, 'splitter': 'random', 'min_weight_fraction_leaf': 0.4555038931381821}. Best is trial 0 with value: 0.006301227937167475.
[I 2022-03-13 23:20:34,596] Trial 3 f

[I 2022-03-13 23:20:35,510] Trial 28 finished with value: 0.014263353725303185 and parameters: {'max_depth': 49, 'min_samples_split': 7, 'min_samples_leaf': 10, 'splitter': 'best', 'min_weight_fraction_leaf': 0.09295167087654435}. Best is trial 22 with value: 0.014864697520817649.
[I 2022-03-13 23:20:35,550] Trial 29 finished with value: 0.00588078292090688 and parameters: {'max_depth': 50, 'min_samples_split': 6, 'min_samples_leaf': 10, 'splitter': 'best', 'min_weight_fraction_leaf': 0.1088962332471908}. Best is trial 22 with value: 0.014864697520817649.
[I 2022-03-13 23:20:35,585] Trial 30 finished with value: 0.008642270554227838 and parameters: {'max_depth': 25, 'min_samples_split': 6, 'min_samples_leaf': 19, 'splitter': 'best', 'min_weight_fraction_leaf': 0.14600101153850167}. Best is trial 22 with value: 0.014864697520817649.
[I 2022-03-13 23:20:35,630] Trial 31 finished with value: 0.012855794861737602 and parameters: {'max_depth': 80, 'min_samples_split': 17, 'min_samples_leaf'

[I 2022-03-13 23:20:36,869] Trial 56 finished with value: 0.013241458755688829 and parameters: {'max_depth': 62, 'min_samples_split': 6, 'min_samples_leaf': 9, 'splitter': 'best', 'min_weight_fraction_leaf': 0.06940083218760884}. Best is trial 33 with value: 0.017269214195454907.
[I 2022-03-13 23:20:36,888] Trial 57 finished with value: -0.00035520424272772466 and parameters: {'max_depth': 75, 'min_samples_split': 9, 'min_samples_leaf': 2, 'splitter': 'random', 'min_weight_fraction_leaf': 0.4976889980519941}. Best is trial 33 with value: 0.017269214195454907.
[I 2022-03-13 23:20:36,942] Trial 58 finished with value: 0.013257439924612813 and parameters: {'max_depth': 52, 'min_samples_split': 14, 'min_samples_leaf': 12, 'splitter': 'best', 'min_weight_fraction_leaf': 0.06023332972586888}. Best is trial 33 with value: 0.017269214195454907.
[I 2022-03-13 23:20:36,968] Trial 59 finished with value: 1.9990867382690958e-05 and parameters: {'max_depth': 57, 'min_samples_split': 32, 'min_sample

[I 2022-03-13 23:20:38,285] Trial 84 finished with value: 0.005739386949379144 and parameters: {'max_depth': 46, 'min_samples_split': 8, 'min_samples_leaf': 11, 'splitter': 'best', 'min_weight_fraction_leaf': 0.11495369970742567}. Best is trial 33 with value: 0.017269214195454907.
[I 2022-03-13 23:20:38,354] Trial 85 finished with value: 0.008839387580360736 and parameters: {'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 10, 'splitter': 'best', 'min_weight_fraction_leaf': 0.034600655704855564}. Best is trial 33 with value: 0.017269214195454907.
[I 2022-03-13 23:20:38,399] Trial 86 finished with value: 0.014457041073578036 and parameters: {'max_depth': 53, 'min_samples_split': 5, 'min_samples_leaf': 9, 'splitter': 'best', 'min_weight_fraction_leaf': 0.09189863078850932}. Best is trial 33 with value: 0.017269214195454907.
[I 2022-03-13 23:20:38,448] Trial 87 finished with value: 0.012223397766432997 and parameters: {'max_depth': 58, 'min_samples_split': 26, 'min_samples_lea

[I 2022-03-13 23:20:39,884] Trial 112 finished with value: 0.013469993830557137 and parameters: {'max_depth': 37, 'min_samples_split': 37, 'min_samples_leaf': 2, 'splitter': 'best', 'min_weight_fraction_leaf': 0.06496709879651932}. Best is trial 33 with value: 0.017269214195454907.
[I 2022-03-13 23:20:39,957] Trial 113 finished with value: 0.015649345084522537 and parameters: {'max_depth': 41, 'min_samples_split': 21, 'min_samples_leaf': 4, 'splitter': 'best', 'min_weight_fraction_leaf': 0.01744755715331335}. Best is trial 33 with value: 0.017269214195454907.
[I 2022-03-13 23:20:39,999] Trial 114 finished with value: 0.00588078292090688 and parameters: {'max_depth': 39, 'min_samples_split': 24, 'min_samples_leaf': 4, 'splitter': 'best', 'min_weight_fraction_leaf': 0.11060183650267258}. Best is trial 33 with value: 0.017269214195454907.
[I 2022-03-13 23:20:40,046] Trial 115 finished with value: 0.014864905034278952 and parameters: {'max_depth': 11, 'min_samples_split': 34, 'min_samples_

...

[I 2022-03-13 23:33:05,920] Trial 4900 finished with value: 0.01774371374815764 and parameters: {'max_depth': 9, 'min_samples_split': 38, 'min_samples_leaf': 14, 'splitter': 'best', 'min_weight_fraction_leaf': 3.669412854917682e-05}. Best is trial 2109 with value: 0.026376412054658238.
[I 2022-03-13 23:33:06,242] Trial 4901 finished with value: 0.013791055245762962 and parameters: {'max_depth': 14, 'min_samples_split': 39, 'min_samples_leaf': 10, 'splitter': 'best', 'min_weight_fraction_leaf': 0.00873416759151141}. Best is trial 2109 with value: 0.026376412054658238.
[I 2022-03-13 23:33:06,554] Trial 4902 finished with value: 0.013770981037989127 and parameters: {'max_depth': 12, 'min_samples_split': 39, 'min_samples_leaf': 16, 'splitter': 'best', 'min_weight_fraction_leaf': 0.022718698893223335}. Best is trial 2109 with value: 0.026376412054658238.
[I 2022-03-13 23:33:06,808] Trial 4903 finished with value: 0.02068302538648692 and parameters: {'max_depth': 10, 'min_samples_split': 39,

[I 2022-03-13 23:33:14,381] Trial 4928 finished with value: 0.018254941256550694 and parameters: {'max_depth': 10, 'min_samples_split': 38, 'min_samples_leaf': 16, 'splitter': 'best', 'min_weight_fraction_leaf': 0.01637340215734535}. Best is trial 2109 with value: 0.026376412054658238.
[I 2022-03-13 23:33:14,686] Trial 4929 finished with value: 0.01248085498712892 and parameters: {'max_depth': 15, 'min_samples_split': 39, 'min_samples_leaf': 16, 'splitter': 'best', 'min_weight_fraction_leaf': 0.007841181676987021}. Best is trial 2109 with value: 0.026376412054658238.
[I 2022-03-13 23:33:14,945] Trial 4930 finished with value: -0.0005455746332481137 and parameters: {'max_depth': 12, 'min_samples_split': 30, 'min_samples_leaf': 17, 'splitter': 'best', 'min_weight_fraction_leaf': 0.4786409812398862}. Best is trial 2109 with value: 0.026376412054658238.
[I 2022-03-13 23:33:15,171] Trial 4931 finished with value: 0.01519107106957085 and parameters: {'max_depth': 10, 'min_samples_split': 39,

[I 2022-03-13 23:33:22,178] Trial 4956 finished with value: 1.9990867383468114e-05 and parameters: {'max_depth': 8, 'min_samples_split': 39, 'min_samples_leaf': 15, 'splitter': 'best', 'min_weight_fraction_leaf': 0.40849011742857255}. Best is trial 2109 with value: 0.026376412054658238.
[I 2022-03-13 23:33:22,414] Trial 4957 finished with value: 0.018812640525321722 and parameters: {'max_depth': 11, 'min_samples_split': 39, 'min_samples_leaf': 16, 'splitter': 'best', 'min_weight_fraction_leaf': 0.008777962267517914}. Best is trial 2109 with value: 0.026376412054658238.
[I 2022-03-13 23:33:22,714] Trial 4958 finished with value: 0.012346690027516427 and parameters: {'max_depth': 17, 'min_samples_split': 39, 'min_samples_leaf': 16, 'splitter': 'best', 'min_weight_fraction_leaf': 0.030290480534293285}. Best is trial 2109 with value: 0.026376412054658238.
[I 2022-03-13 23:33:23,016] Trial 4959 finished with value: 0.0203921418411962 and parameters: {'max_depth': 9, 'min_samples_split': 38,

[I 2022-03-13 23:33:29,844] Trial 4984 finished with value: 0.010710620970895257 and parameters: {'max_depth': 16, 'min_samples_split': 26, 'min_samples_leaf': 16, 'splitter': 'best', 'min_weight_fraction_leaf': 0.008117858507661906}. Best is trial 2109 with value: 0.026376412054658238.
[I 2022-03-13 23:33:30,109] Trial 4985 finished with value: 0.0017844614274871162 and parameters: {'max_depth': 13, 'min_samples_split': 26, 'min_samples_leaf': 16, 'splitter': 'best', 'min_weight_fraction_leaf': 0.2916576233587192}. Best is trial 2109 with value: 0.026376412054658238.
[I 2022-03-13 23:33:30,343] Trial 4986 finished with value: 0.013742685721036652 and parameters: {'max_depth': 12, 'min_samples_split': 27, 'min_samples_leaf': 16, 'splitter': 'best', 'min_weight_fraction_leaf': 0.022832816421867463}. Best is trial 2109 with value: 0.026376412054658238.
[I 2022-03-13 23:33:30,646] Trial 4987 finished with value: 0.018257784094252383 and parameters: {'max_depth': 11, 'min_samples_split': 2

Best params:  {'max_depth': 9, 'min_samples_split': 37, 'min_samples_leaf': 4, 'splitter': 'best', 'min_weight_fraction_leaf': 0.00952179564895864}


### Use the best parameters to train the Decision Tree model

In [29]:
regr = DecisionTreeRegressor(max_depth=9, min_samples_split=37, min_samples_leaf=4,
                             splitter='best', min_weight_fraction_leaf=0.00952179564895864)
regr = regr.fit(X_train, y_train)

# predict on test set values
y_pred = regr.predict(X_test)

# evaluate MSE error
MSE = mean_squared_error(y_test, y_pred)
accuracy = regr.score(X_test, y_test)
print('mean square error: ', MSE)
print('R2 score: ', r2_score(y_test, y_pred))


mean square error:  0.13712743603812122
R2 score:  0.026376412054658127


### Do hyperparamter tuning to optimize the training Random Forest model

In [13]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    max_depth = trial.suggest_int('max_depth', 2, 80)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 40)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    min_weight_fraction_leaf = trial.suggest_uniform(
        'min_weight_fraction_leaf', 0.0, 0.5)
    # get the score for the hyperparameters chosen
    regr = RandomForestRegressor(n_estimators=n_estimators,max_depth=max_depth, 
                                  min_samples_split=min_samples_split,
                                 min_samples_leaf=min_samples_leaf,
                                 min_weight_fraction_leaf=min_weight_fraction_leaf)
    regr.fit(X_train, y_train)
    if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return regr.score(X_test, y_test)

study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize')
study.optimize(objective, n_trials=5000)
print('Best params: ', study.best_params)

[I 2022-03-13 15:31:58,442] A new study created in memory with name: no-name-3bd8b63a-467f-45dc-ae0d-53332323086f
[I 2022-03-13 15:31:58,822] Trial 0 finished with value: 0.0007429989810968651 and parameters: {'n_estimators': 72, 'max_depth': 34, 'min_samples_split': 39, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0.40024970975531027}. Best is trial 0 with value: 0.0007429989810968651.
[I 2022-03-13 15:32:01,196] Trial 1 finished with value: 0.01753503959818903 and parameters: {'n_estimators': 85, 'max_depth': 28, 'min_samples_split': 7, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0.020010426272441484}. Best is trial 1 with value: 0.01753503959818903.
[I 2022-03-13 15:32:01,546] Trial 2 finished with value: 0.0008275745320747152 and parameters: {'n_estimators': 76, 'max_depth': 54, 'min_samples_split': 26, 'min_samples_leaf': 8, 'min_weight_fraction_leaf': 0.3416637551280257}. Best is trial 1 with value: 0.01753503959818903.
[I 2022-03-13 15:32:01,917] Trial 3 finished wi

[I 2022-03-13 15:32:16,902] Trial 28 finished with value: 0.005211642798145122 and parameters: {'n_estimators': 24, 'max_depth': 45, 'min_samples_split': 23, 'min_samples_leaf': 13, 'min_weight_fraction_leaf': 0.18974410471284756}. Best is trial 22 with value: 0.04064348758147207.
[I 2022-03-13 15:32:17,576] Trial 29 finished with value: 0.00777532421852134 and parameters: {'n_estimators': 39, 'max_depth': 35, 'min_samples_split': 35, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0764940627837428}. Best is trial 22 with value: 0.04064348758147207.
[I 2022-03-13 15:32:20,627] Trial 30 finished with value: 0.04048946179784885 and parameters: {'n_estimators': 65, 'max_depth': 60, 'min_samples_split': 9, 'min_samples_leaf': 7, 'min_weight_fraction_leaf': 0.0022838758969253737}. Best is trial 22 with value: 0.04064348758147207.
[I 2022-03-13 15:32:22,568] Trial 31 finished with value: 0.017287751441452115 and parameters: {'n_estimators': 67, 'max_depth': 58, 'min_samples_split': 8, '

[I 2022-03-13 15:33:10,670] Trial 56 finished with value: 0.006557633809517904 and parameters: {'n_estimators': 85, 'max_depth': 53, 'min_samples_split': 6, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.09637232928605578}. Best is trial 43 with value: 0.051306906738883606.
[I 2022-03-13 15:33:12,108] Trial 57 finished with value: 0.009263606032753446 and parameters: {'n_estimators': 81, 'max_depth': 27, 'min_samples_split': 20, 'min_samples_leaf': 9, 'min_weight_fraction_leaf': 0.059714261177925006}. Best is trial 43 with value: 0.051306906738883606.
[I 2022-03-13 15:33:13,387] Trial 58 finished with value: 0.01801420186112046 and parameters: {'n_estimators': 46, 'max_depth': 68, 'min_samples_split': 6, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0.01695462355813512}. Best is trial 43 with value: 0.051306906738883606.
[I 2022-03-13 15:33:13,773] Trial 59 finished with value: 0.0007936350755689547 and parameters: {'n_estimators': 88, 'max_depth': 46, 'min_samples_split': 3

[I 2022-03-13 15:34:01,590] Trial 84 finished with value: 0.004723967271474505 and parameters: {'n_estimators': 67, 'max_depth': 3, 'min_samples_split': 17, 'min_samples_leaf': 20, 'min_weight_fraction_leaf': 0.048106818292900186}. Best is trial 43 with value: 0.051306906738883606.
[I 2022-03-13 15:34:03,348] Trial 85 finished with value: 0.020016198305413013 and parameters: {'n_estimators': 59, 'max_depth': 25, 'min_samples_split': 14, 'min_samples_leaf': 18, 'min_weight_fraction_leaf': 0.012295115588215666}. Best is trial 43 with value: 0.051306906738883606.
[I 2022-03-13 15:34:04,556] Trial 86 finished with value: 0.013883879964818324 and parameters: {'n_estimators': 56, 'max_depth': 33, 'min_samples_split': 18, 'min_samples_leaf': 17, 'min_weight_fraction_leaf': 0.03593440322261032}. Best is trial 43 with value: 0.051306906738883606.
[I 2022-03-13 15:34:05,165] Trial 87 finished with value: 0.00471279990628326 and parameters: {'n_estimators': 78, 'max_depth': 21, 'min_samples_split

[I 2022-03-13 15:34:49,547] Trial 112 finished with value: 0.04039057555218328 and parameters: {'n_estimators': 47, 'max_depth': 47, 'min_samples_split': 13, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0003304381619545066}. Best is trial 43 with value: 0.051306906738883606.
[I 2022-03-13 15:34:51,881] Trial 113 finished with value: 0.015762504848155334 and parameters: {'n_estimators': 95, 'max_depth': 36, 'min_samples_split': 27, 'min_samples_leaf': 16, 'min_weight_fraction_leaf': 0.022861966284449205}. Best is trial 43 with value: 0.051306906738883606.
[I 2022-03-13 15:34:53,096] Trial 114 finished with value: 0.011392985857772375 and parameters: {'n_estimators': 59, 'max_depth': 39, 'min_samples_split': 18, 'min_samples_leaf': 13, 'min_weight_fraction_leaf': 0.040614915138355134}. Best is trial 43 with value: 0.051306906738883606.
[I 2022-03-13 15:34:54,938] Trial 115 finished with value: 0.020529145919574776 and parameters: {'n_estimators': 63, 'max_depth': 32, 'min_samples

...

[I 2022-03-13 20:31:05,702] Trial 4900 finished with value: 0.05130122698233153 and parameters: {'n_estimators': 100, 'max_depth': 25, 'min_samples_split': 16, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 4.963970782686216e-05}. Best is trial 3566 with value: 0.06145400758567554.
[I 2022-03-13 20:31:09,383] Trial 4901 finished with value: 0.024404671240248033 and parameters: {'n_estimators': 95, 'max_depth': 27, 'min_samples_split': 15, 'min_samples_leaf': 7, 'min_weight_fraction_leaf': 0.008560762620685102}. Best is trial 3566 with value: 0.06145400758567554.
[I 2022-03-13 20:31:12,528] Trial 4902 finished with value: 0.01972709214473023 and parameters: {'n_estimators': 97, 'max_depth': 31, 'min_samples_split': 24, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.01685503790957337}. Best is trial 3566 with value: 0.06145400758567554.
[I 2022-03-13 20:31:17,748] Trial 4903 finished with value: 0.05011140147138349 and parameters: {'n_estimators': 93, 'max_depth': 22, 'min_samp

[I 2022-03-13 20:32:42,947] Trial 4928 finished with value: 0.016298192330767458 and parameters: {'n_estimators': 96, 'max_depth': 32, 'min_samples_split': 14, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0.023185063076336297}. Best is trial 3566 with value: 0.06145400758567554.
[I 2022-03-13 20:32:48,047] Trial 4929 finished with value: 0.05093839393187727 and parameters: {'n_estimators': 92, 'max_depth': 24, 'min_samples_split': 12, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 8.4300250231319e-05}. Best is trial 3566 with value: 0.06145400758567554.
[I 2022-03-13 20:32:51,428] Trial 4930 finished with value: 0.02396930539627473 and parameters: {'n_estimators': 95, 'max_depth': 25, 'min_samples_split': 23, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.009549364402991279}. Best is trial 3566 with value: 0.06145400758567554.
[I 2022-03-13 20:32:54,465] Trial 4931 finished with value: 0.016938747518837838 and parameters: {'n_estimators': 98, 'max_depth': 27, 'min_sampl

[I 2022-03-13 20:34:10,068] Trial 4956 finished with value: 0.018044519414330495 and parameters: {'n_estimators': 90, 'max_depth': 19, 'min_samples_split': 12, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0.017090671367424492}. Best is trial 3566 with value: 0.06145400758567554.
[I 2022-03-13 20:34:12,965] Trial 4957 finished with value: 0.015947475067229933 and parameters: {'n_estimators': 97, 'max_depth': 23, 'min_samples_split': 10, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0.02407790520209974}. Best is trial 3566 with value: 0.06145400758567554.
[I 2022-03-13 20:34:13,680] Trial 4958 finished with value: 0.00082535209848833 and parameters: {'n_estimators': 96, 'max_depth': 24, 'min_samples_split': 16, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.4137937623602446}. Best is trial 3566 with value: 0.06145400758567554.
[I 2022-03-13 20:34:17,013] Trial 4959 finished with value: 0.022971221348638382 and parameters: {'n_estimators': 92, 'max_depth': 33, 'min_sample

[I 2022-03-13 20:35:45,814] Trial 4984 finished with value: 0.012746641439684403 and parameters: {'n_estimators': 92, 'max_depth': 31, 'min_samples_split': 13, 'min_samples_leaf': 9, 'min_weight_fraction_leaf': 0.03577226908462059}. Best is trial 3566 with value: 0.06145400758567554.
[I 2022-03-13 20:35:46,572] Trial 4985 finished with value: 0.0008024804472600566 and parameters: {'n_estimators': 98, 'max_depth': 21, 'min_samples_split': 24, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0.3296005458992015}. Best is trial 3566 with value: 0.06145400758567554.
[I 2022-03-13 20:35:50,029] Trial 4986 finished with value: 0.023027742325916645 and parameters: {'n_estimators': 95, 'max_depth': 23, 'min_samples_split': 38, 'min_samples_leaf': 14, 'min_weight_fraction_leaf': 0.009263431966246119}. Best is trial 3566 with value: 0.06145400758567554.
[I 2022-03-13 20:35:52,696] Trial 4987 finished with value: 0.016266636551060643 and parameters: {'n_estimators': 89, 'max_depth': 20, 'min_sam

Best params:  {'n_estimators': 99, 'max_depth': 23, 'min_samples_split': 14, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 2.9893745932052474e-05}


### Use the best parameters to train the Random Forest model

In [27]:
# Define a random forest classifier object
rand = RandomForestRegressor(n_estimators=99, max_depth=23, min_samples_split=14, 
                              min_samples_leaf=3, min_weight_fraction_leaf=2.9893745932052474e-05)

# Fit to your training data
rand = rand.fit(X_train, y_train)

# How accurate is your random forest estimator?
y_pred = rand.predict(X_test)

# Evaluate MSE error
MSE = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('mean square error: ',MSE)
print('R2 score: ', r2)

mean square error:  0.13080424329963283
R2 score:  0.07127194703429207
